This job loads all resources from the transparenz.hamburg.de website, translates it into a dataframe and shows the biggest files on the website (not possible to do this over the web page directly)

Can be used on every other website which is using CKAN as well, e.g. the European Data Portal.

http://suche.transparenz.hamburg.de/

https://www.europeandataportal.eu/data/en/ (doesn't work since they are using an old version and size='None' 20170217 tma)

https://data.gov.uk/data/search


In [10]:
## Example from http://docs.ckan.org/en/ckan-2.2.3/api.html#making-an-api-request
## transparenz.hamburg.de is based on CKAN

import json
import pandas as pd
from pandas import DataFrame
from pandas.io.json import json_normalize

#import urllib2
#py 3 http://stackoverflow.com/questions/2792650/python3-error-import-error-no-module-name-urllib2
from urllib.request import urlopen
#import urllib
#py 3 http://stackoverflow.com/questions/31827012/python-importing-urllib-quote
import urllib.parse
import json
import pprint

dfj = DataFrame()

    ##
    ## When limit is disabled, won't work.With offset and paging it does, putting things in smaller bits
    ##

my_offset = 0
offset_step = 250
still_has_data = True

while still_has_data:
    print('Offset: ' + str(my_offset))
    # Use the json module to dump a dictionary to a string for posting.
    data_string = urllib.parse.quote(json.dumps({'id': 'data-explorer', 'limit': offset_step, 'offset': my_offset}))

    #py 3 http://stackoverflow.com/questions/13175253/python-error-type-error-post-data-should-be-bytes-also-user-agent-issue
    data_string = data_string.encode('ascii')

    # Make the HTTP request.
    #response = urllib2.urlopen('http://suche.transparenz.hamburg.de/api/action/group_list',
    #        data_string)
    # py 3
    response = urlopen('http://suche.transparenz.hamburg.de/api/action/current_package_list_with_resources', data_string)
    assert response.code == 200

    # Use the json module to load CKAN's response into a dictionary.
    response_dict = json.loads(response.read())

    # Check the contents of the response.
    assert response_dict['success'] is True
    result = response_dict['result']
    #pprint.pprint(result)

    if (len(response_dict['result']) > 0):
        df = json_normalize(response_dict['result'])
    
        for item in df['resources']:
            # Check if there is any data and if the list has items
            if (item != None and len(item) > 0):
                dfj_worker = json_normalize(item)
                dfj = dfj.append(dfj_worker, ignore_index=True)
    
        print(dfj.shape)
        my_offset = my_offset + offset_step
    else:
        still_has_data = False

print('Done after offset ' + str(my_offset))

Offset: 0
(360, 27)
Offset: 250
(651, 27)
Offset: 500
(919, 27)
Offset: 750
(1196, 27)
Offset: 1000
(1529, 27)
Offset: 1250
(1870, 27)
Offset: 1500
(2237, 27)
Offset: 1750
(2601, 27)
Offset: 2000
(2971, 27)
Offset: 2250
(3379, 27)
Offset: 2500
(3761, 27)
Offset: 2750
(4092, 27)
Offset: 3000
(4479, 27)
Offset: 3250
(4870, 27)
Offset: 3500
(5285, 27)
Offset: 3750
(5674, 27)
Offset: 4000
(6016, 27)
Offset: 4250
(6397, 27)
Offset: 4500
(6802, 27)
Offset: 4750
(7216, 27)
Offset: 5000
(7553, 27)
Offset: 5250
(7940, 27)
Offset: 5500
(8360, 27)
Offset: 5750
(8755, 27)
Offset: 6000
(9138, 27)
Offset: 6250
(9461, 27)
Offset: 6500
(9893, 27)
Offset: 6750
(10280, 27)
Offset: 7000
(10668, 27)
Offset: 7250
(11027, 27)
Offset: 7500
(11397, 27)
Offset: 7750
(11769, 27)
Offset: 8000
(12109, 27)
Offset: 8250
(12480, 27)
Offset: 8500
(12805, 27)
Offset: 8750
(13221, 27)
Offset: 9000
(13598, 27)
Offset: 9250
(13954, 27)
Offset: 9500
(14375, 27)
Offset: 9750
(14834, 27)
Offset: 10000
(15234, 27)
Offset: 10

Hamburg:

Offset: 64500
(96938, 27)
Offset: 64750
Done after offset 64750

In [25]:
dfj.to_csv('transparenz-hamburg-ressources.csv')

In [ ]:
dfj.dtypes

In [134]:
dfj[['file_size','size']] = dfj[['file_size','size']].apply(pd.to_numeric)
dfj.dtypes

In [136]:
dfj['human_size'] = dfj['size'] / 1024/1024/1024
dfj[['size', 'format', 'name', 'human_size']].sort_values('size', ascending=False)

,size,format,name,human_size
5134,15972052378,ZIP,"Download DOP 20cm, März/April 2014 (14,8 GB, JPG)",1.487513e+01
5135,15840717308,ZIP,"Download DOP 20cm, April/Juni 2015 (14,7 GB, JPG)",1.475282e+01
8334,6242290610,ZIP,Download ALKIS Liegenschaftskarte - ausgewählt...,5.813586e+00
14490,6242290610,jpg,Download ALKIS Liegenschaftskarte - ausgewählt...,5.813586e+00
6103,6242290610,ZIP,Download ALKIS Liegenschaftskarte - ausgewählt...,5.813586e+00
8330,5635219857,ZIP,Download ALKIS Liegenschaftskarte - ausgewählt...,5.248207e+00
14503,5635219857,jpg,Download ALKIS Liegenschaftskarte - ausgewählt...,5.248207e+00
8341,5612032142,ZIP,Download ALKIS Liegenschaftskarte - ausgewählt...,5.226612e+00
14497,5612032142,jpg,Download ALKIS Liegenschaftskarte - ausgewählt...,5.226612e+00
6094,5612032142,ZIP,Download ALKIS Liegenschaftskarte - ausgewählt...,5.226612e+00


In [138]:
# Save to Excel
writer = pd.ExcelWriter(
    'Transparenz_Hamburg_Resources.xlsx', engine='xlsxwriter')
dfj.to_excel(writer, 'Tabelle1')
writer.save()

D:\Development\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:830: UserWarning: Ignoring URL 'http://geodienste.hamburg.de/HH_WFS_RISE_FG?service=WFS&request=GetFeature&version=1.1.0&TypeName=FD_FHH_Map_BSU_WSB2_RISE_FG:Soziale_Stadt,FD_FHH_Map_BSU_WSB2_RISE_FG:Aktive_Stadt_und_Ortsteilzentren,FD_FHH_Map_BSU_WSB2_RISE_FG:Hamburgische_Landesfoerderung,FD_FHH_Map_BSU_WSB2_RISE_FG:Staedtebauliche_Sanierung,FD_FHH_Map_BSU_WSB2_RISE_FG:Stadtumbau_West' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
D:\Development\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:830: UserWarning: Ignoring URL 'http://geodienste.hamburg.de/HH_WFS_PflegeuGesundheit_BAE?service=WFS&request=GetFeature&VERSION=1.1.0&typename=hh_bae_gesundheit_aerzte,hh_bae_gesundheit_apotheken,hh_bae_gesundheit_seniorentreff,hh_bae_gesundheit_krankenkassen,hh_bae_gesundheit_krankenhaeuser,hh_bae_gesundheit_ambpfldienst,hh_bae_gesundheit_sportubewegung,hh_bae_gesun

This listing didn't include csv files. Might be connected to the limit (was 500 initially), but the job now already takes a while. 

So why not search?

My results (see checks below): Search makes it hard to search with OR (which it can't) or to sort DESC (which it can't) so still neccessary to load all ressources like up there (takes an hour or two)

In [18]:
# Use the json module to dump a dictionary to a string for posting.
data_string = urllib.parse.quote(json.dumps({'id': 'data-explorer', 'query': 'format:csv'}))#, 'order_by': 'size', 'limit': 50}))

#py 3 http://stackoverflow.com/questions/13175253/python-error-type-error-post-data-should-be-bytes-also-user-agent-issue
data_string = data_string.encode('ascii')


# Make the HTTP request.
#response = urllib2.urlopen('http://suche.transparenz.hamburg.de/api/action/group_list',
#        data_string)
# py 3
#response = urlopen('http://suche.transparenz.hamburg.de/api/action/resource_search', data_string)
#response = urlopen('https://www.europeandataportal.eu/data/en/api/action/current_package_list_with_resources', data_string)
response = urlopen('https://data.gov.uk/api/action/resource_search', data_string)

assert response.code == 200

# Use the json module to load CKAN's response into a dictionary.
response_dict = json.loads(response.read())

# Check the contents of the response.
assert response_dict['success'] is True
result = response_dict['result']

dfs = json_normalize(response_dict['result'])

dfs

,count,results
0,42309,[{'resource_group_id': '000073ca-2fb3-4829-806...


In [19]:
dfs.dtypes

dflist = DataFrame()

for listentry in dfs['results']:
    dflist_worker = json_normalize(listentry)
    dflist = dflist.append(dflist_worker)

dflist

,Valid From,Valid To,active,archiver,cache_last_updated,cache_url,coverage_end_date,coverage_start_date,created,csvlint_json,...,state,temporal_coverage_from,temporal_coverage_to,title,type,uri,url,url_type,webstore_last_updated,webstore_url
0,NaN,NaN,NaN,NaN,None,None,NaN,NaN,2015-12-15T20:21:12.435957,NaN,...,active,NaN,NaN,NaN,NaN,NaN,http://data.jncc.gov.uk/data/edf4070e-a301-46d...,None,None,None
1,NaN,NaN,NaN,NaN,2016-03-07T13:34:24.364235,None,NaN,NaN,2016-03-01T11:35:11.564013,"{""in_progress"": false, ""data"": {""num_errors"": ...",...,active,04/01/2016,None,NaN,NaN,NaN,http://opendata.leeds.gov.uk/downloads/floodin...,None,None,None
2,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,NaN,...,active,NaN,NaN,NaN,NaN,NaN,http://www.hscic.gov.uk/catalogue/PUB05054/att...,None,None,None
3,NaN,Ongoing,NaN,NaN,None,None,NaN,NaN,2016-02-04T09:37:09.860153,NaN,...,active,NaN,NaN,NaN,NaN,NaN,https://data.yorkopendata.org/dataset/b1914559...,None,None,None
4,NaN,NaN,True,NaN,None,None,NaN,NaN,2016-11-01T13:50:20,NaN,...,active,NaN,NaN,Learning Disability Service Users 2015-16,NaN,NaN,http://opendata.cambridgeshireinsight.org.uk/f...,None,None,None
5,NaN,NaN,NaN,NaN,None,None,NaN,NaN,2016-09-30T23:35:12.751466,NaN,...,active,NaN,NaN,NaN,NaN,NaN,https://data.gov.uk/sites/default/files/organo...,None,None,None
6,NaN,NaN,NaN,NaN,None,None,NaN,NaN,2016-10-19T12:22:50.606265,NaN,...,active,NaN,NaN,NaN,NaN,NaN,https://data.gov.uk/sites/default/files/organo...,None,None,None
7,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,NaN,...,active,NaN,NaN,NaN,NaN,NaN,http://www.ordnancesurvey.co.uk/docs/purchase-...,None,None,None
8,NaN,NaN,NaN,NaN,None,None,NaN,NaN,2016-04-22T16:14:23.699237,NaN,...,active,NaN,NaN,NaN,NaN,NaN,http://www.trafford.gov.uk/opendata/sets/suppl...,None,None,None
9,NaN,NaN,NaN,NaN,None,None,NaN,NaN,2015-12-15T20:20:37.438708,NaN,...,active,NaN,NaN,NaN,NaN,NaN,http://data.jncc.gov.uk/data/240d7f13-589c-4fc...,None,None,None


In [20]:
dflist.dtypes

Valid From                   object
Valid To                     object
active                       object
archiver                     object
cache_last_updated           object
cache_url                    object
coverage_end_date            object
coverage_start_date          object
created                      object
csvlint_json                 object
datastore_active             object
datastore_json               object
date                         object
description                  object
download_url                 object
filename                     object
format                       object
hash                         object
hub-id                       object
id                           object
last_modified                object
lazyboy_json                 object
mimetype                     object
mimetype_inner               object
name                         object
position                      int64
publish-date                 object
qa                          

In [22]:
#dflist[['file_size','size']] = dflist[['file_size','size']].apply(pd.to_numeric)
dflist[['size']] = dflist[['size']].apply(pd.to_numeric)
dflist.dtypes

Valid From                    object
Valid To                      object
active                        object
archiver                      object
cache_last_updated            object
cache_url                     object
coverage_end_date             object
coverage_start_date           object
created                       object
csvlint_json                  object
datastore_active              object
datastore_json                object
date                          object
description                   object
download_url                  object
filename                      object
format                        object
hash                          object
hub-id                        object
id                            object
last_modified                 object
lazyboy_json                  object
mimetype                      object
mimetype_inner                object
name                          object
position                       int64
publish-date                  object
q

In [32]:
dflists = dflist.sort_values('size',ascending=False)
dflists[['name', 'format', 'url', 'size']]



,name,format,url,size
6393,2014-15,CSV,https://files.datapress.com/calderdale/dataset...,137254694.0
8148,2013-14,CSV,https://files.datapress.com/calderdale/dataset...,136083841.0
10202,2012-13,CSV,https://files.datapress.com/calderdale/dataset...,132378955.0
11813,2011-12,CSV,https://files.datapress.com/calderdale/dataset...,131401011.0
13055,2010-11,CSV,https://files.datapress.com/calderdale/dataset...,125999192.0
3013,06/05/2016,CSV,https://aql.datapress.com/leeds/dataset/househ...,70703578.0
2784,2016,CSV,https://aql.datapress.com/leeds/dataset/counci...,65089424.0
3878,2016,CSV,https://aql.datapress.com/leeds/dataset/leeds-...,46880221.0
6155,Raw data 2015-17,CSV,https://files.datapress.com/barnet/dataset/par...,40742543.0
6563,2015,CSV,https://aql.datapress.com/leeds/dataset/leeds-...,37925184.0


In [33]:
for itera in dflists['url'].head(5):
    print(itera)

https://files.datapress.com/calderdale/dataset/customer-meter-data/2016-08-31T14:17:44/201415 YW Customer Meter Data.csv
https://files.datapress.com/calderdale/dataset/customer-meter-data/2016-08-31T14:44:23/2013-14.csv
https://files.datapress.com/calderdale/dataset/customer-meter-data/2016-08-31T14:56:47/2012-13.csv
https://files.datapress.com/calderdale/dataset/customer-meter-data/2016-08-31T15:03:55/2011-12.csv
https://files.datapress.com/calderdale/dataset/customer-meter-data/2016-08-31T15:16:34/2010-11.csv
